In [ ]:
#define time series
p_ROI = "cc400"

In [ ]:
#connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install PyPrind
import pandas as pd
import numpy as np
import os
import pyprind
import numpy.ma as ma
import pickle

In [ ]:
#make file names
def get_key(filename):
    f_split = filename.split('_')
    if f_split[3] == 'rois':
        key = '_'.join(f_split[0:3]) 
    else:
        key = '_'.join(f_split[0:2])
    return key

In [ ]:
#load time series from files
data_main_path = 'drive/My Drive/Downloaded/Outputs/cpac/filt_global/rois_'+p_ROI
flist = os.listdir(data_main_path)
print(len(flist))

for f in range(len(flist)):
    flist[f] = get_key(flist[f])
    

df_labels = pd.read_csv('https://raw.githubusercontent.com/preprocessed-connectomes-project/abide/master/Phenotypic_V1_0b_preprocessed1.csv')#path 

df_labels.DX_GROUP = df_labels.DX_GROUP.map({1: 1, 2:0})
print(len(df_labels))

labels = {}
for row in df_labels.iterrows():
    file_id = row[1]['FILE_ID']
    y_label = row[1]['DX_GROUP']
    if file_id == 'no_filename':
        continue
    assert(file_id not in labels)
    labels[file_id] = y_label


884
1112


In [ ]:
s_labels = {}
for row in df_labels.iterrows():
    file_id = row[1]['FILE_ID']
    sex_label = row[1]['SEX']
    if file_id == 'no_filename':
        continue
    assert(file_id not in s_labels)
    s_labels[file_id] = sex_label

a_labels = {}
for row in df_labels.iterrows():
    file_id = row[1]['FILE_ID']
    age_lable = row[1]['AGE_AT_SCAN']
    if file_id == 'no_filename':
        continue
    assert(file_id not in a_labels)
    a_labels[file_id] = age_lable

In [ ]:
# s_labels['Caltech_0051456']
a_labels['Caltech_0051456']

55.4

In [ ]:
def get_label(filename):
    assert (filename in labels)
    return labels[filename]

def get_s_label(filename):
    assert (filename in s_labels)
    return s_labels[filename]

def get_a_label(filename):
    assert (filename in a_labels)
    return a_labels[filename]

def get_corr_data(filename):
    for file in os.listdir(data_main_path):
        if file.startswith(filename):
            df = pd.read_csv(os.path.join(data_main_path, file), sep='\t')
            
    with np.errstate(invalid="ignore"):
        corr = np.nan_to_num(np.corrcoef(df.T))
        mask = np.invert(np.tri(corr.shape[0], k=-1, dtype=bool))
        m = ma.masked_where(mask == 1, mask)
        return ma.masked_where(m, corr).compressed()

def get_corr_matrix(filename):
    for file in os.listdir(data_main_path):
        if file.startswith(filename):
            df = pd.read_csv(os.path.join(data_main_path, file), sep='\t')
    with np.errstate(invalid="ignore"):
        corr = np.nan_to_num(np.corrcoef(df.T))
        return corr

In [ ]:
#make corr matrix
if not os.path.exists('./correlations_file'+p_ROI+'.pkl'):
    pbar=pyprind.ProgBar(len(flist))
    all_corr = {}
    for f in flist:
      
        lab = get_label(f)
        s_lab = get_s_label(f)
        a_lab = get_a_label(f)
        all_corr[f] = (get_corr_data(f), lab, s_lab, a_lab)
        pbar.update()

    print('Corr-computations finished')

    pickle.dump(all_corr, open('./correlations_file'+p_ROI+'.pkl', 'wb'))
    print('Saving to file finished')

else:
    all_corr = pickle.load(open('./correlations_file'+p_ROI+'.pkl', 'rb'))

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:03:59


Corr-computations finished
Saving to file finished


In [ ]:
np.shape(all_corr['Caltech_0051464'][0])

(76636,)

In [ ]:
# !cp '/content/correlations_filecc400.pkl' '/content/drive/My Drive/correlation_matrix_sa'